In [1]:
from pyspark.sql import SparkSession
# from pyspark import StorageLevel
# import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, LongType
from pyspark.sql import SparkSession, SQLContext , functions
from pyspark.sql.functions import array_contains,count,countDistinct,col, udf
# from shapely.geometry import Point
# from shapely.geometry import Polygon

# from sedona.register import SedonaRegistrator
# from sedona.core.SpatialRDD import SpatialRDD
# from sedona.core.SpatialRDD import PointRDD
# from sedona.core.SpatialRDD import PolygonRDD
# from sedona.core.SpatialRDD import LineStringRDD
# from sedona.core.enums import FileDataSplitter
# from sedona.utils.adapter import Adapter
# from sedona.core.spatialOperator import KNNQuery
# from sedona.core.spatialOperator import JoinQuery
# from sedona.core.spatialOperator import JoinQueryRaw
# from sedona.core.spatialOperator import RangeQuery
# from sedona.core.spatialOperator import RangeQueryRaw
# from sedona.core.formatMapper.shapefileParser import ShapefileReader
# from sedona.core.formatMapper import WkbReader
# from sedona.core.formatMapper import WktReader
# from sedona.core.formatMapper import GeoJsonReader
# from sedona.sql.types import GeometryType
# from sedona.core.enums import GridType
# from sedona.core.SpatialRDD import RectangleRDD
# from sedona.core.enums import IndexType
# from sedona.core.geom.envelope import Envelope
# from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [2]:
spark = SparkSession.builder \
.appName('ReadFromParquet') \
.master('local[*]') \
.getOrCreate()

In [3]:
spark

In [4]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [5]:
sqlContext.setConf("spark.sql.parquet.binaryAsString","true")
osmWay = sqlContext.read.parquet("./rawdata/china-latest.osm.pbf.way.parquet")
osmWay = osmWay.select('id','tags','nodes')
osmWay.createOrReplaceTempView("osmWay")

In [6]:
osmWay.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- nodes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- index: integer (nullable = true)
 |    |    |-- nodeId: long (nullable = true)



In [7]:
osmWay.head()

Row(id=4181592, tags=[Row(key='name', value='港鐵機場快綫 MTR Airport Express'), Row(key='gauge', value='1432'), Row(key='layer', value='1'), Row(key='bridge', value='yes'), Row(key='name:en', value='MTR Airport Express'), Row(key='name:zh', value='港鐵機場快綫'), Row(key='network', value='MTR'), Row(key='railway', value='subway'), Row(key='voltage', value='1500'), Row(key='operator', value='香港鐵路有限公司 MTR Corporation'), Row(key='frequency', value='0'), Row(key='start_date', value='1998-07-06'), Row(key='electrified', value='contact_line')], nodes=[Row(index=0, nodeId=253333910), Row(index=1, nodeId=3304026514)])

In [8]:
osmWay.show(5)

+-------+--------------------------+--------------------+
|     id|                      tags|               nodes|
+-------+--------------------------+--------------------+
|4181592|[[name, 港鐵機場快綫 MT...|[[0, 253333910], ...|
|4181898|    [[brand, 港鐵 MTR],...|[[0, 357211075], ...|
|4182605|      [[leisure, park],...|[[0, 24355560], [...|
|4182662| [[alt_name, 大嶼山及機...|[[0, 288378656], ...|
|4187007|      [[fee, yes], [foo...|[[0, 3049050712],...|
+-------+--------------------------+--------------------+
only showing top 5 rows



In [9]:
osmWay.count()

8301264

In [10]:
osmNode = sqlContext.read.parquet("./rawdata/china-latest.osm.pbf.node.parquet")
osmNode = osmNode.select('id','tags', 'latitude','longitude')
osmNode.createOrReplaceTempView("osmNode")

In [11]:
osmNode.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [12]:
osmNode.head()

Row(id=274901, tags=[], latitude=22.3460861, longitude=114.18111400000001)

In [13]:
osmRelation = sqlContext.read.parquet("./rawdata/china-latest.osm.pbf.relation.parquet")
osmRelation = osmRelation.select('id','tags', 'members')
osmRelation.createOrReplaceTempView("osmRelation")

In [14]:
osmRelation.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- members: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- role: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [15]:
osmRelation.head()

Row(id=2664, tags=[Row(key='name', value='文晖路'), Row(key='type', value='street'), Row(key='name:en', value='Wenhui Road'), Row(key='name:zh', value='文晖路'), Row(key='name:zh_pinyin', value='Wénhuī Lù')], members=[Row(id=4317650, role='', type='Way'), Row(id=572561555, role='', type='Way'), Row(id=577190913, role='', type='Way'), Row(id=572561552, role='', type='Way'), Row(id=332083690, role='', type='Way'), Row(id=16755045, role='', type='Way'), Row(id=558188950, role='', type='Way'), Row(id=4402796, role='', type='Way'), Row(id=572561557, role='', type='Way'), Row(id=4402802, role='', type='Way'), Row(id=577190914, role='', type='Way'), Row(id=4402803, role='', type='Way'), Row(id=558188951, role='', type='Way'), Row(id=4402797, role='', type='Way')])

In [16]:
osmRelation.show(5)

+-----+-----------------------+--------------------+
|   id|                   tags|             members|
+-----+-----------------------+--------------------+
| 2664|[[name, 文晖路], [ty...|[[4317650, , Way]...|
|13422|   [[type, multipoly...|[[24426608, outer...|
|13423|   [[code, BHA426010...|[[822446817, oute...|
|13424|   [[ele, 2033.0], [...|[[24426368, inner...|
|13425|   [[code, BHA42A601...|[[827786405, oute...|
+-----+-----------------------+--------------------+
only showing top 5 rows



In [17]:
# Spark 3.x
# osmData = spark.read.format("osm.pbf").load("./rawdata/china-latest.osm.pbf")
# osmData.head()

In [18]:
# osmWay.write\
# .options(catalog=catalog)\
# .format("org.apache.spark.sql.execution.datasources.hbase")\
# .save()

In [19]:
def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string, StringType())

osmWay = osmWay.withColumn('tags_array', array_to_string_udf(osmWay["tags"]))
osmWay = osmWay.withColumn('nodes_array', array_to_string_udf(osmWay["nodes"]))

osmWay.drop("tags").drop("nodes").write.csv('./csvData/osmWay.csv',header=True)

In [20]:
spark.read.csv("./csvData/osmWay.csv", header=True).show(5)

+---------+--------------------+--------------------+
|       id|          tags_array|         nodes_array|
+---------+--------------------+--------------------+
|825713816|[Row(key='buildin...|[Row(index=0, nod...|
|825713817|[Row(key='buildin...|[Row(index=0, nod...|
|825713818|[Row(key='buildin...|[Row(index=0, nod...|
|825713819|[Row(key='buildin...|[Row(index=0, nod...|
|825713820|[Row(key='buildin...|[Row(index=0, nod...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [23]:
osmNode = osmNode.withColumn('tags_array', array_to_string_udf(osmNode["tags"]))

df = osmNode.drop("tags")
df.write.csv('./csvData/osmNode.csv',header=True)

Py4JJavaError: An error occurred while calling o136.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.s

In [ ]:
spark.read.csv("./csvData/osmNode.csv", header=True).show(5)

In [ ]:
osmRelation = osmRelation.withColumn('tags_array', array_to_string_udf(osmRelation["tags"]))
osmRelation = osmRelation.withColumn('members_array', array_to_string_udf(osmRelation["members"]))

osmRelation.drop("tags").drop("members").write.csv('./csvData/osmRelation.csv',header=True)

In [ ]:
spark.read.csv("./csvData/osmRelation.csv", header=True).show(5)